## Transfer Learning with Convolutional Neural Networks for Hydrological Streamline Detection

**Authors**: Nattapon Jaroenchai <sup>a, b</sup>, Shaowen Wang <sup>a, b, *</sup>, Lawrence V. Stanislawski <sup>c</sup>, Ethan Shavers <sup>c</sup>, E. Lynn Usery <sup>c</sup>, Shaohua Wang <sup>a, b</sup>, Sophie Wang, and Li Chen <sup>a, b</sup>  

<sup>a</sup> *Department of Geography and Geographic Information Science, University of Illinois at Urbana-Champaign, Urbana, IL, USA*  
<sup>b</sup> *CyberGIS Center for Advanced Digital and Spatial Studies, University of Illinois at Urbana-Champaign, Urbana, IL, USA*  
<sup>c</sup> *U.S. Geology Survey, Center of Excellence for Geospatial Information Science, Rolla, MO, USA*  
<sup>d</sup> *School of Geoscience and Info-Physics, Central South University, Changsha, Hunan, China*  

Last Updated Date: July 27, 2022

### Abstract 

Streamline network delineation plays a vital role in various scientific disciplines and business applications, such as agriculture sustainability, river dynamics, wetland inventory, watershed analysis, surface water management, and flood mapping. Traditionally, flow accumulation techniques have been used to extract streamlines, which delineate streamline primarily based on topological information. Recently, machine learning techniques such as the U-net model have been applied for streamlining delineation. Even though the model shows promising performance in geographic areas that it has been trained on, its performance drops significantly when applied to other areas. In this paper, we apply a transfer learning approach in which we use the pre-trained network architectures that have been trained on a large dataset, ImageNet. Then, we fine-tuned the neural networks using smaller datasets collected from Rowan Creek and Covington areas in the US. When we compared the models pre-trained on ImageNet with an attention U-net model which are fine-tuned on the Rowan Creek area, we found that the DenseNet169 model achieved an F1-score of 85% which is about 4% higher than the attention U-net model. Then, to compare the transferability of the models, the top three models in Rowan Creek area and the attention U-net model were fine-tuned further with the samples from the Covington area. We were able to achieve an F1-score of 71.87% in predicting the steam pixels in the Covington area which is significantly higher than training the model from scratch with the samples collected from the Covington area and slightly higher than the attention U-net model.

### Keywords: 

Transfer Learning, Convolutional neural network, Remote Sensing, Streamline detection



## Table of content

1. Introduction (This notebook)
 - Study areas 
 - Input data
2. Create Base Models in Rowan Creekc (Base_model_training.ipynb)
3. Transfer Rowan Creek Models to Convington (Transfer_source_to_target.ipynb)
4. Model Evaluation (Modle_evaluation.ipynb)


## 1. Introduction 

Streamline network delineation is essential for various applications, such as sustainable agriculture, river dynamics, watershed analysis, etc. The recent advancement in machine learning, in particular deep learning, led to the development of an attention U-net model which outperforms traditional methods for streamline delineation. However, the performance of the model drops significantly when being applied to different geographic areas.  
   
  
Transfer learning is a branch of machine learning which leverages knowledge from one domain to another and promises to overcome the challenges of streamline delineation across different geographies. 

#### 1.1. Study Areas

<img src="notebook_data/study_areas.png" width=100% height=100% />


##### Rowan Creek, NC
- Area of around 18.11km2
- Virtually flat with about 222 meters elevation variation 
- Land cover dominated by forest, wetlands, and agricultural land   
  
#### Covington River, VA
- Area of around 108km2
- High elevation variation from 125 to 1,040 meters
- Land cover dominated by forest and agricultural land


### 1.2 Input Data

<img src="notebook_data/Input_rasters.png" width=100% height=100% />


Eight co-registered 1-m resolution Lidar and elevation-derived raster data layers were used for training, validation, and testing in the research. The 1-m raster layers include:   
a) digital elevation model (DEM);   
b) geometric curvature determined from the DEM (Sangireddy et al., 2016);   
c) slope derived from the DEM;   
d) mean nadir angle positive openness derived from the DEM using a 10-cell radius with 32 directions (Doneus, 2013);   
e) topographic position index (TPI) derived from the DEM using a 21-cell by 21-cell window;   
f) return intensity determined from the Lidar ground points averaged with inverse distance weighting using 10 nearest points;   
g) geomorphon type estimated as one of ten common landforms derived from the DEM using a 10-cell radius (Jasiewicz and Stepinski, 2013);  
h) TPI derived from the DEM using a 3-cell by 3-cell window 







